In [2]:
import numpy as np
import pystan
import matplotlib.pyplot as plt
import random
import time
%matplotlib inline

In [3]:
def logit(z): return 1./(1.+np.exp(-z))

In [4]:
def generate_players(num_of_players, skill_cap):
    ## returns a list of player's skill
    return [round(random.uniform(0,skill_cap),2) for i in range(num_of_players)]


def generate_games(players, num_of_games, scale=0.3, style='pystan'):

    if style == 'pystan':
        
        player1=[]
        player2=[]
        outcome=[]
        for i in range(num_of_games):
            p1,p2 = random.sample(range(len(players)),2)
            win_rate=logit(scale*(players[p1]-players[p2]) )

            ##pystan player id is ONE based
            player1.append(p1+1)
            player2.append(p2+1)
            
            outcome.append(*random.choices([1,0],weights=[win_rate,1-win_rate]))

        return np.array([player1,player2,outcome])
    
    elif style == 'pygm':
        games=[]
        for i in range(num_of_games):
            p1,p2 = random.sample(range(len(players)),2)
            win_rate=logit(scale*(players[p1]-players[p2]) )
            
            games.append((p1,p2,*random.choices([1,-1],weights=[win_rate,1-win_rate])))
        return games
    
    assert False 


In [5]:
def prediction_accuracy(skill_data, samples, true_players, num_valid_game=1000, scale=0.3):
    
    valid_games = generate_games(true_players, num_valid_game, scale, style='pystan')

    acc=0
    for g in range(num_valid_game):
        i,j,result=valid_games[0][g],valid_games[1][g],valid_games[2][g]

        prediction = logit( skill_data['scale']*(samples['skill'][:,i-1]-samples['skill'][:,j-1]) ).mean()

        acc+= int( (prediction >= 0.5 and result==1) or (prediction<0.5 and result==0) )
    return acc/num_valid_game

def prediction_accuracy_2(skill_data, samples, valid_data, scale=0.3):
    
    #valid_games = generate_games(true_players, num_valid_game, scale, style='pystan')

    acc=0
    for g in range(valid_data.shape[1]):
        p1,p2,result=valid_data[0][g],valid_data[1][g],valid_data[2][g]

        win_rate = logit( skill_data['scale']*(samples['skill'][:,p1-1]-samples['skill'][:,p2-1]) ).mean()
        
        predict_result = random.choices([1,0],weights=[win_rate,1-win_rate])

        if predict_result == result:
            acc += 1
    return acc/valid_data.shape[1]

In [6]:
skill_model = """
data {
  int<lower=1> N;             # Total number of players
  int<lower=1> E;             # number of games
  real<lower=0> scale;        # scale value for probability computation
  int<lower=0,upper=1> win[E]; # PA wins vs PB
  int PA[E];                  # player info between each game
  int PB[E];                  # 
}
parameters {
  vector [N] skill;           # skill values for each player
}

model{
  for (i in 1:N){ skill[i]~normal(0,10); }
  for (i in 1:E){
    win[i] ~ bernoulli_logit( (scale)*(skill[PA[i]]-skill[PB[i]]) );
  }   # win probability is a logit function of skill difference
}
"""

Now, compile the model.  We'll save a version of it so we don't have to recompile every time.

In [7]:
import pickle
try:     # load it if already compiled
    sm = pickle.load(open('skill_model.pkl', 'rb'))
except:  # ow, compile and save compiled model
    sm = pystan.StanModel(model_code = skill_model)
    with open('skill_model.pkl', 'wb') as f: pickle.dump(sm, f)

We also need the observed data: number of players and games, which pairs played each game, and who won:

Now, we can perform MCMC on the model, and extract the samples:

If we just want the mean estimate for each player's skill level, just take the empirical average over the samples:

In [ ]:
def run_data(nplayer):
    scale=0.5
    num_of_players=nplayer
    result={}
    #Generate games 
    print("start generating")
    Data = []
    for trials in range(10):
        p = generate_players(num_of_players,50)
        games = generate_games(p,3000, scale, style='pystan')#####
        games_pool = games[:,:2500]
        valid_games = games[:,2500:]
        Data.append((p,games_pool,valid_games))


    for num_of_games in [3,5,10,20,35,50,70,100,500,700,1000,2500]:
        starting_num_of_games=time.time()
        trials_result=[]
        print("starting num_of_games =",num_of_games)
        for trials in range(10):
            true_players = Data[trials][0]

            train_games = Data[trials][1][:,:num_of_games]
            print("true skills:",true_players)
            skill_data = {
                'N': num_of_players,
                'E': num_of_games,
                'scale': scale,###
                'win':train_games[2],
                'PA': train_games[0],
                'PB': train_games[1]
            }
            starting=time.time()
            fit = sm.sampling(data=skill_data, iter=10000, chains=4)####
            print("spent:",round(time.time()-starting,3))
            samples = fit.extract()
            print("predicted skills:",samples['skill'].mean(0))
            acc=prediction_accuracy_2(skill_data,samples,Data[trials][2])
            print("acc:",acc,"\n")
            trials_result.append(acc)

        result[num_of_games]=trials_result
        print("Complete",num_of_games,"| Spent:",round(time.time()-starting_num_of_games,2))
    return result
